# YOLO
The goal of this notebook is to do transfer learning of Fishial´s already implemented YOLO model. We do this by using Ultralytics

In [29]:
from ultralytics import YOLO
import os
import requests
import logging
from zipfile import ZipFile
import pandas as pd
import torch
import cv2
import matplotlib.pyplot as plt
import copy
from IPython.display import Image, display

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [30]:
DATA_DIR = "../data_csv/"
timor_leste_data_path = os.path.join(DATA_DIR, "timor-leste.csv") # Annotation info for ground truth
images_path = "../data_images/"

In [31]:
import re

def extract_and_format_species(big_string):
    # Find all genus_species patterns (ignore numbers)
    matches = re.findall(r"\b([a-z]+_[a-z]+)\b", big_string)

    formatted = []
    for species in matches:
        genus, epithet = species.split("_")
        formatted.append(f"{genus.capitalize()} {epithet}")

    return formatted

big_string = """
acanthocybium_solandri	
29
alectis_ciliaris	
15
amblygaster_sirm	
1
aphareus_rutilans	
3
arius_arius	
1
aurigequula_fasciata	
70
carangoides_praeustus	
1
caranx_ignobilis	
41
caranx_lugubris	
1
caranx_melampygus	
9
caranx_sexfasciatus	
1
caranx_tille	
1
chirocentrus_dorab	
81
chirocentrus_nudus	
2
decapterus_macarellus	
52
decapterus_macrosoma	
4
elagatis_bipinnulata	
8
elagatos_bipinnulata	
1
epinephelus_maculatus	
19
epinephelus_radiatus	
6
etelis_carbunculus	
16
gazza_minuta	
59
gymnocranius_grandoculis	
38
katsuwonus_pelamis	
10
lethrinus_erythracanthus	
2
lethrinus_mahsena	
13
lethrinus_obsoletus	
1
lethrinus_ornatus	
1
lethrinus_reticulatus	
1
lujanus_gibbus	
51
lutjanus_bohar	
3
lutjanus_fulviflamma	
27
lutjanus_fulvus	
47
lutjanus_gibbus	
3
lutjanus_johnii	
68
lutjanus_kasmira	
10
lutjanus_rivulatus	
1
lutjanus_rufolineatus	
2
lutjanus_russellii	
7
monotaxis_grandoculis	
7
mulloidichtys_vanicolensis	
75
neoniphon_sammara	
15
parupeneus_heptacanthus	
24
parupeneus_indicus	
13
platax_boersii	
1
plectorhinchus_vittatus	
2
pomacanthus_annularis	
1
pomadasys_argenteus	
33
pomadasys_kaakan	
46
priacanthus_sagittarius	
1
pristigenys_niphonia	
15
psettodes_erumei	
3
pseudobalistes_flavimarginatus	
8
rastrelliger_kanagurta	
129
sardinella_albella	
16
scarus_quoyi	
2
scolopsis_vosmeri	
4
scomberoides_lysan	
1
scomberoides_tol	
106
scomberomorus_commerson	
28
seriola_dumerili	
1
siganus_spinus	
1
upeneus_vittatus	
77
variola_albimarginata	
7
"""

relevant_species = extract_and_format_species(big_string)

print(relevant_species)
print(f"Total relevant species: {len(relevant_species)}")

['Acanthocybium solandri', 'Alectis ciliaris', 'Amblygaster sirm', 'Aphareus rutilans', 'Arius arius', 'Aurigequula fasciata', 'Carangoides praeustus', 'Caranx ignobilis', 'Caranx lugubris', 'Caranx melampygus', 'Caranx sexfasciatus', 'Caranx tille', 'Chirocentrus dorab', 'Chirocentrus nudus', 'Decapterus macarellus', 'Decapterus macrosoma', 'Elagatis bipinnulata', 'Elagatos bipinnulata', 'Epinephelus maculatus', 'Epinephelus radiatus', 'Etelis carbunculus', 'Gazza minuta', 'Gymnocranius grandoculis', 'Katsuwonus pelamis', 'Lethrinus erythracanthus', 'Lethrinus mahsena', 'Lethrinus obsoletus', 'Lethrinus ornatus', 'Lethrinus reticulatus', 'Lujanus gibbus', 'Lutjanus bohar', 'Lutjanus fulviflamma', 'Lutjanus fulvus', 'Lutjanus gibbus', 'Lutjanus johnii', 'Lutjanus kasmira', 'Lutjanus rivulatus', 'Lutjanus rufolineatus', 'Lutjanus russellii', 'Monotaxis grandoculis', 'Mulloidichtys vanicolensis', 'Neoniphon sammara', 'Parupeneus heptacanthus', 'Parupeneus indicus', 'Platax boersii', 'Ple

In [32]:
df_tl_ann = pd.read_csv(timor_leste_data_path, encoding="utf-8-sig", header=0, skiprows=1)

exclude_species = [
    "Ferdauia orthagrammus",
    "Lutjanus timoriensis",
    "Myripristis sp.",
    "Scolopsis lineata",
    "Caranx sp.",
    "Leiognathus sp.",
    "Sargocentron sp."
]

# Filter rows where Species_name IS in the excluded list
marked_df = df_tl_ann[df_tl_ann["Species_name"].isin(exclude_species)]

# Get unique image filenames that contain those species
marked_images = marked_df["image_file"].unique().tolist()

print("Number of images containing excluded species:", len(marked_images))
print(marked_images)



Number of images containing excluded species: 32
['1689557063983.jpg', '1689557783158.jpg', '1689578033635.jpg', '1689743281308.jpg', '1690158839295.jpg', '1690890877921.jpg', '1692068612306.jpg', '1692330774500.jpg', '1692367560146.jpg', '1692581111734.jpg', '1692674979299.jpg', '1692680823804.jpg', '1692707991167.jpg', '1692957477379.jpg', '1693380383471.jpg', '1693546616778.jpg', '1693549450702.jpg', '1694346888884.jpg', '1694483777242.jpg', '1695889981671.jpg', '1695890043610.jpg', '1697009354475.jpg', '1697011438775.jpg', '1697012535972.jpg', '1697272759438.jpg', '1697506294036.jpg', '1697506536349.jpg', '1698239567947.jpg', '1698548955693.jpg', '1698885483705.jpg', '1699355005032.jpg', '1699581757679.jpg']


In [33]:
import shutil
# ---------------- PATHS ----------------
images_path = "../data_images/"            # Source folder
output_path = "../filtered_images/"        # New folder to create

# Create the folder if not exists
os.makedirs(output_path, exist_ok=True)

# ---------------- LOAD ANNOTATIONS ----------------
df_tl_ann = pd.read_csv(timor_leste_data_path, encoding="utf-8-sig", header=0, skiprows=1)
df_tl_ann = df_tl_ann[["image_file", "Species_name"]]

# ---------------- FILTER BY RELEVANT SPECIES ----------------
df_filtered = df_tl_ann[df_tl_ann["Species_name"].isin(relevant_species)]

# ---------------- KEEP ONLY EXISTING FILES ----------------
existing_files = set(os.listdir(images_path))
df_filtered = df_filtered[df_filtered["image_file"].isin(existing_files)]

# ---------------- COPY RELEVANT FILES ----------------
copied = 0
for filename in df_filtered["image_file"]:
    src = os.path.join(images_path, filename)
    dst = os.path.join(output_path, filename)
    shutil.copy(src, dst)
    copied += 1

# ---------------- SUMMARY ----------------
print("Total annotated images:", len(df_tl_ann))
print("Relevant species rows:", len(df_filtered))
print("Files copied:", copied)
print("Copied into:", output_path)

Total annotated images: 603
Relevant species rows: 295
Files copied: 295
Copied into: ../filtered_images/


In [34]:
import os

folder_path = "../filtered_images/"   # change this to your folder

# Count files only (ignores subfolders)
num_files = len([f for f in os.listdir(folder_path) 
                 if os.path.isfile(os.path.join(folder_path, f))])

print("Number of files in folder:", num_files)


Number of files in folder: 247


In [35]:
unique_image_files = df_filtered["image_file"].nunique()
total_rows = len(df_filtered)

print("Rows:", total_rows)
print("Unique image files:", unique_image_files)
print("Duplicates:", total_rows - unique_image_files)


Rows: 295
Unique image files: 247
Duplicates: 48


In [22]:
def on_train_epoch_end(trainer):
    print(f"Finished epoch {trainer.epoch}/{trainer.epochs}")

model = YOLO("yolo11n.pt")

model.add_callback("on_train_epoch_end", on_train_epoch_end)

# Train the model
train_results = model.train(
    data="final_dataset/data.yaml",
    epochs=300,
    imgsz=640,
    device="cuda",
    patience=30
)

# Evaluate the model's performance on the validation set
metrics = model.val()

New https://pypi.org/project/ultralytics/8.3.237 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.235  Python-3.12.10 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=final_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False,

error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\torch\utils\data\_utils\worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\ultralytics\data\base.py", line 374, in __getitem__
    return self.transforms(self.get_image_and_label(index))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\ultralytics\data\base.py", line 387, in get_image_and_label
    label["img"], label["ori_shape"], label["resized_shape"] = 

# Prediction

In [71]:
# ============================================
# YOLOv11 – Run on TRAIN set, export boxes,
# build image-level predictions, and
# generate confusion matrix
# ============================================

import os
import pandas as pd
import matplotlib.pyplot as plt

from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# ----------------------------
# CONFIG
# ----------------------------
MODEL_PATH = "runs/detect/train10/weights/best.pt"
DATASET_ROOT = "final_dataset"
TRAIN_IMG_DIR = os.path.join(DATASET_ROOT, "test/images")
TRAIN_LABEL_DIR = os.path.join(DATASET_ROOT, "test/labels")

IMG_EXT = ".jpg"   # change if png/jpeg
CONF_THRES = 0.25
IMG_SIZE = 640
CM_OUT = "confusion_matrix_train.png"

# ----------------------------
# 1. Load model & run inference
# ----------------------------
model = YOLO(MODEL_PATH)

results = model.predict(
    source=TRAIN_IMG_DIR,
    imgsz=IMG_SIZE,
    conf=CONF_THRES,
    save=True,
    save_txt=True,
    save_conf=True
)

# ----------------------------
# 2. Collect bounding boxes
# ----------------------------
rows = []

for r in results:
    if r.boxes is None:
        continue

    image_name = os.path.basename(r.path)

    for box in r.boxes:
        rows.append({
            "image": image_name,
            "pred_class": int(box.cls),
            "confidence": float(box.conf),
            "x1": float(box.xyxy[0][0]),
            "y1": float(box.xyxy[0][1]),
            "x2": float(box.xyxy[0][2]),
            "y2": float(box.xyxy[0][3]),
        })

df_pred_boxes = pd.DataFrame(rows)
df_pred_boxes.to_csv("train_predictions_boxes.csv", index=False)

# ----------------------------
# 3. Image-level prediction
#    (highest confidence box)
# ----------------------------
df_pred_img = (
    df_pred_boxes
    .sort_values("confidence", ascending=False)
    .groupby("image")
    .first()
    .reset_index()
)

# ----------------------------
# 4. Load ground truth labels
# ----------------------------
gt_rows = []

for file in os.listdir(TRAIN_LABEL_DIR):
    if not file.endswith(".txt"):
        continue

    with open(os.path.join(TRAIN_LABEL_DIR, file)) as f:
        lines = f.readlines()

    if len(lines) == 0:
        continue

    gt_rows.append({
        "image": file.replace(".txt", IMG_EXT),
        "gt_class": int(lines[0].split()[0])
    })

df_gt = pd.DataFrame(gt_rows)

# ----------------------------
# 5. Merge GT & predictions
# ----------------------------
df_eval = df_gt.merge(
    df_pred_img[["image", "pred_class"]],
    on="image",
    how="inner"
)

df_eval.to_csv("train_image_level_eval.csv", index=False)

# ----------------------------
# 6. Confusion matrix (SAVE)
# ----------------------------
cm = confusion_matrix(
    df_eval["gt_class"],
    df_eval["pred_class"]
)

fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(cm)
disp.plot(ax=ax, cmap="Blues", xticks_rotation=45)

ax.set_title("YOLOv11 – Train Set Confusion Matrix")
plt.tight_layout()
plt.savefig(CM_OUT, dpi=300)
plt.close(fig)

print(f"Confusion matrix saved to: {CM_OUT}")

# ----------------------------
# 7. Classification report
# ----------------------------
print(
    classification_report(
        df_eval["gt_class"],
        df_eval["pred_class"],
        digits=3
    )
)

# ----------------------------
# 8. Summary
# ----------------------------
print(f"Images evaluated: {len(df_eval)}")
print(f"Total predicted boxes: {len(df_pred_boxes)}")



image 1/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689120590238_jpg.rf.8b16027a2f429b42a32fc4d2728df1b2.jpg: 640x480 (no detections), 27.7ms
image 2/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689254343092_jpg.rf.073389a363bea13f9d20992c03716813.jpg: 640x480 (no detections), 6.3ms
image 3/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689388044927_jpg.rf.865dfbfcf6491f32506638a0bc60d7ac.jpg: 480x640 1 aurigequula_fasciata, 7.3ms
image 4/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689388326411_jpg.rf.a934e3368903e2b1b59a04f0c48aa8a5.jpg: 480x640 1 pomadasys_kaakan, 7.2ms
image 5/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689557098876_jpg.rf.8c7b9f0026b7ec74139aca00d62974f2.jpg: 640x480 5 chirocentrus_dorabs, 6.3ms
image 6/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689577829534_jpg.rf.12d0188

c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [89]:
# ============================================================
# YOLOv11 – FULL PIPELINE
# Run on TRAIN set, collect detections,
# filter species, and SAVE a normalized
# confusion matrix (paper-style)
# ============================================================

import os
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report

# ----------------------------
# CONFIG
# ----------------------------
MODEL_PATH = "runs/detect/run_200_epochs/weights/best.pt"
DATASET_ROOT = "final_dataset"

TRAIN_IMG_DIR = os.path.join(DATASET_ROOT, "test/images")
TRAIN_LABEL_DIR = os.path.join(DATASET_ROOT, "test/labels")
DATA_YAML = os.path.join(DATASET_ROOT, "data.yaml")

IMG_EXT = ".jpg"      # change if needed
IMG_SIZE = 640
CONF_THRES = 0.25

CM_OUT = "confusion_matrix_train_normalized50.png"

exclude_species = [
    "Ferdauia orthagrammus",
    "Lutjanus timoriensis",
    "Myripristis sp.",
    "Scolopsis lineata",
    "Caranx sp.",
    "Leiognathus sp.",
    "Sargocentron sp."
]

# ----------------------------
# 1. Load class names
# ----------------------------
with open(DATA_YAML, "r") as f:
    data_yaml = yaml.safe_load(f)

class_names = data_yaml["names"]  # id -> name

# ----------------------------
# 2. Load model & run inference
# ----------------------------
model = YOLO(MODEL_PATH)

results = model.predict(
    source=TRAIN_IMG_DIR,
    imgsz=IMG_SIZE,
    conf=CONF_THRES,
    save=True,        # saves images with boxes
    save_txt=True,    # YOLO-format .txt files
    save_conf=True    # include confidence
)

# ----------------------------
# 3. Collect bounding boxes
# ----------------------------
rows = []

for r in results:
    if r.boxes is None:
        continue

    image_name = os.path.basename(r.path)

    for box in r.boxes:
        rows.append({
            "image": image_name,
            "pred_class": int(box.cls),
            "confidence": float(box.conf)
        })

df_pred_boxes = pd.DataFrame(rows)

# ----------------------------
# 4. Image-level prediction
#    (highest confidence box)
# ----------------------------
df_pred_img = (
    df_pred_boxes
    .sort_values("confidence", ascending=False)
    .groupby("image")
    .first()
    .reset_index()
)

# ----------------------------
# 5. Load ground truth labels
# ----------------------------
gt_rows = []

for file in os.listdir(TRAIN_LABEL_DIR):
    if not file.endswith(".txt"):
        continue

    with open(os.path.join(TRAIN_LABEL_DIR, file)) as f:
        lines = f.readlines()

    if len(lines) == 0:
        continue

    gt_rows.append({
        "image": file.replace(".txt", IMG_EXT),
        "gt_class": int(lines[0].split()[0])
    })

df_gt = pd.DataFrame(gt_rows)

# ----------------------------
# 6. Merge GT & predictions
# ----------------------------
df_eval = df_gt.merge(
    df_pred_img,
    on="image",
    how="inner"
)

df_eval["gt_name"] = df_eval["gt_class"].apply(lambda i: class_names[i])
df_eval["pred_name"] = df_eval["pred_class"].apply(lambda i: class_names[i])

# ----------------------------
# 7. Exclude unwanted species
# ----------------------------
df_eval = df_eval[
    ~df_eval["gt_name"].isin(exclude_species)
]

# ----------------------------
# Images with correct prediction
# ----------------------------
df_match = df_eval[df_eval["gt_name"] == df_eval["pred_name"]]

matched_images = df_match["image"].tolist()

print(f"Number of matched images: {len(matched_images)}")
print("First 10 matched images:", matched_images[:10])


# ----------------------------
# 8. Confusion matrix (normalized)
# ----------------------------
labels = sorted(df_eval["gt_name"].unique())

cm = confusion_matrix(
    df_eval["gt_name"],
    df_eval["pred_name"],
    labels=labels
).astype(float)

row_sums = cm.sum(axis=1, keepdims=True)
cm_norm = np.divide(cm, row_sums, where=row_sums != 0)

# ----------------------------
# 9. Plot & save matrix
# ----------------------------
fig, ax = plt.subplots(figsize=(14, 12))

sns.heatmap(
    cm_norm,
    ax=ax,
    cmap="Blues",
    annot=True,
    fmt=".1f",
    xticklabels=labels,
    yticklabels=labels,
    linewidths=0.5,
    linecolor="gray",
    cbar_kws={"label": "Fraction normalized by ground truth"}
)

ax.set_xlabel("Prediction")
ax.set_ylabel("Ground truth")
ax.set_title("YOLOv11 (200 epochs)– Test Set Confusion Matrix (Normalized)")

plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)

plt.tight_layout()
plt.savefig(CM_OUT, dpi=300)
plt.close(fig)

print(f"Saved normalized confusion matrix to: {CM_OUT}")

# ----------------------------
# 10. Optional report
# ----------------------------
print(
    classification_report(
        df_eval["gt_name"],
        df_eval["pred_name"],
        digits=3
    )
)

# ----------------------------
# 11. Summary
# ----------------------------
print(f"Images evaluated: {len(df_eval)}")
print(f"Classes shown: {len(labels)}")



image 1/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689120590238_jpg.rf.8b16027a2f429b42a32fc4d2728df1b2.jpg: 640x480 (no detections), 11.0ms
image 2/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689254343092_jpg.rf.073389a363bea13f9d20992c03716813.jpg: 640x480 (no detections), 9.1ms
image 3/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689388044927_jpg.rf.865dfbfcf6491f32506638a0bc60d7ac.jpg: 480x640 1 aurigequula_fasciata, 10.9ms
image 4/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689388326411_jpg.rf.a934e3368903e2b1b59a04f0c48aa8a5.jpg: 480x640 1 pomadasys_kaakan, 7.6ms
image 5/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689557098876_jpg.rf.8c7b9f0026b7ec74139aca00d62974f2.jpg: 640x480 5 chirocentrus_dorabs, 10.9ms
image 6/226 c:\Users\kienn\Downloads\Prosjektoppgave\yolo\final_dataset\test\images\1689577829534_jpg.rf.12d01

c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\kienn\Downloads\Prosjektoppgave\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [81]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(
    df_eval["gt_name"],
    df_eval["pred_name"]
)

print(f"Image-level accuracy: {accuracy:.4f}")


Image-level accuracy: 0.0192
